In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [3]:
kospi_downloaded = pd.read_pickle('history/kospi_download_result.pkl')
kospi = pd.read_csv('metadata/kospi.csv', error_bad_lines=False)

b'Skipping line 437: expected 12 fields, saw 13\n'


In [4]:
kospi_downloaded[0:2]

,종목코드,상장일,결과,기업명
0,095570,2015-08-21,Naver,AJ네트웍스
1,068400,2012-07-27,Naver,AJ렌터카


In [5]:
kospi[0:2]

,번호,종목코드,기업명,업종코드,업종,상장주식수(주),자본금(원),액면가(원),통화구분,대표전화,주소,총카운트
0,1,095570,AJ네트웍스,147603.0,산업용 기계 및 장비 임대업,"46,822,295","46,822,295,000","1,000",원(KRW),02-6363-9999,"서울특별시 송파구 정의로8길 9 (문정동,AJ빌딩)",789.0
1,2,068400,AJ렌터카,147601.0,운송장비 임대업,"22,146,300","11,073,150,000",500,원(KRW),1544-1600,서울특별시 구로구 서부샛길 822,789.0


In [35]:
def load_data_by_ticker(ticker) : 
    metadata = kospi[kospi['종목코드'] == ticker]
    display(metadata)
    
    data = pd.read_pickle('data/kospi/%s.data'%ticker)
    data.index = pd.to_datetime(data.index)
    
    data['High'] = pd.to_numeric(data['High'])
    data['Low'] = pd.to_numeric(data['Low'])
    data['Open'] = pd.to_numeric(data['Open'])
    data['Close'] = pd.to_numeric(data['Close'])
    data['Volume'] = pd.to_numeric(data['Volume'])
    display(data.head())
    
    return metadata, data

In [9]:
metadata, data = load_data_by_ticker('095570')

,번호,종목코드,기업명,업종코드,업종,상장주식수(주),자본금(원),액면가(원),통화구분,대표전화,주소,총카운트
0,1,095570,AJ네트웍스,147603.0,산업용 기계 및 장비 임대업,"46,822,295","46,822,295,000","1,000",원(KRW),02-6363-9999,"서울특별시 송파구 정의로8길 9 (문정동,AJ빌딩)",789.0


,High,Low,Open,Close,Volume,Adj Close
2015-08-21,7300,5600,6180,7100,1994022,
2015-08-24,6940,6240,6700,6480,392831,
2015-08-25,6730,6189,6610,6220,242730,
2015-08-26,7760,6260,6260,7090,703721,
2015-08-27,8060,6960,7239,7570,499913,


# Backtesting.py

In [20]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA

/usr/local/lib/python3.6/dist-packages/backtesting/_plotting.py:37: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `bokeh.io.reset_output()`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [23]:
GOOG.head()

,Open,High,Low,Close,Volume
2004-08-19,100.00,104.06,95.96,100.34,22351900
2004-08-20,101.01,109.08,100.50,108.31,11428600
2004-08-23,110.75,113.48,109.05,109.40,9137200
2004-08-24,111.24,111.60,103.57,104.87,7631300
2004-08-25,104.96,108.00,103.88,106.00,4598900


In [27]:
class SmaCross(Strategy) :
    def init(self) :
        Close = self.data.Close
        self.ma1 = self.I(SMA, Close, 10)
        self.ma2 = self.I(SMA, Close, 20)
        
    def next(self) :
        if crossover(self.ma1, self.ma2) :
            self.buy()
        elif crossover(self.ma2, self.ma1) :
            self.sell()

In [37]:
CJCGV.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3669 entries, 2004-12-24 to 2019-10-25
Data columns (total 6 columns):
High         3669 non-null int64
Low          3669 non-null int64
Open         3669 non-null int64
Close        3669 non-null int64
Volume       3669 non-null int64
Adj Close    3669 non-null object
dtypes: int64(5), object(1)
memory usage: 200.6+ KB


In [38]:
CJCGV_meta, CJCGV = load_data_by_ticker('079160')

,번호,종목코드,기업명,업종코드,업종,상장주식수(주),자본금(원),액면가(원),통화구분,대표전화,주소,총카운트
8,9,079160,CJ CGV,105901.0,"영화, 비디오물, 방송프로그램 제작 및 배급업","21,161,313","10,580,656,500",500,원(KRW),02-371-6660,서울특별시 용산구 한강대로23길 55 아이파크몰 6층(한강로동),789.0


,High,Low,Open,Close,Volume,Adj Close
2004-12-24,33100,28500,30000,31400,4039871,
2004-12-27,32300,30500,32100,31500,508141,
2004-12-28,31450,29900,31400,30500,364932,
2004-12-29,31500,30100,30200,31200,203763,
2004-12-30,32000,31050,31200,32000,153647,


In [39]:
bt = Backtest(CJCGV, SmaCross, cash=10000, commission=.002)

In [40]:
bt.run()

Start                     2004-12-24 00:00:00
End                       2019-10-25 00:00:00
Duration                   5418 days 00:00:00
Exposure [%]                          99.2986
Equity Final [$]                      705.936
Equity Peak [$]                       10646.4
Return [%]                           -92.9406
Buy & Hold Return [%]                 9.87261
Max. Drawdown [%]                    -93.4434
Avg. Drawdown [%]                    -1.99608
Max. Drawdown Duration       10 days 00:00:00
Avg. Drawdown Duration        9 days 00:00:00
# Trades                                  197
Win Rate [%]                          33.5025
Best Trade [%]                        74.7655
Worst Trade [%]                      -21.0733
Avg. Trade [%]                      -0.919285
Max. Trade Duration         142 days 00:00:00
Avg. Trade Duration          28 days 00:00:00
Expectancy [%]                        6.78628
SQN                                  -2.10266
Sharpe Ratio                      

In [41]:
bt.plot()